In [2]:
import os
import sys

PROJECT_ROOT = '/'

sys.path.append(PROJECT_ROOT)

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [3]:
conf = {'groups_desc': 'data/small_group_description.csv',
    'train_x': 'data/transactions_train.csv',
    'test_x': 'data/transactions_test.csv',
    'train_target': 'data/train_target.csv',
    'test_target': 'data/test.csv'}

In [4]:
from data_utils import *

In [5]:
%%time

ds = DataSource(conf)
ds.read_data()
ds.validation_split()

Reading groups_desc...
Reading train_x...
Reading test_x...
Reading train_target...
Reading test_target...
Validation split: by clientID
CPU times: user 12.8 s, sys: 6.13 s, total: 18.9 s
Wall time: 22.5 s


In [83]:
print(ds)

Dataset: groups_desc, shape: (204, 2)
Dataset: train_x, shape: (21295142, 4)
Dataset: test_x, shape: (17667328, 4)
Dataset: train_target, shape: (24145, 1)
Dataset: test_target, shape: (20000, 1)
Dataset: valid_x, shape: (5155435, 4)
Dataset: valid_target, shape: (5855, 2)


In [6]:
train_target = ds.get_data('train_target')
train_target.set_index('client_id', inplace=True)

In [7]:
train_x = ds.get_data('train_x')
print(train_x.head())


   client_id  trans_date  small_group  amount_rur
0       3694          26            1      10.006
1       3694          29            3      54.955
2       3694          31            1      10.945
3       3694          34           36      61.721
4       3694          36           25       4.579


In [203]:
gr6 = train_x.groupby(['client_id']).apply(lambda x: pd.Series({'s6':
                    (x['small_group']==169).sum()}))

#gr.set_index('client_id', inplace=True)
joined = train_target.join(gr6)
zero =  joined.groupby('bins').agg({'s6':'mean'})
print(zero.head())

            s6
bins          
0     0.015202
1     0.007810
2     0.000329
3     0.017969


In [8]:
train_x['month_number'] = train_x['trans_date']//30

In [10]:
#print(train_target.head())
print(train_target.join(train_x))
#train_x.groupby(['client_id'])["trans_date"].agg({"tr_c": "count"}).tr_c.max()


           bins  client_id  trans_date  small_group  amount_rur  month_number
client_id                                                                    
1046          0       3694         710           80      30.362            23
34089         2      34445         669            9     101.208            22
34848         1      32728         502           24     665.480            16
47076         3      19350          48           11      22.486             1
10938         2       6767         593            1      82.963            19
...         ...        ...         ...          ...         ...           ...
36716         2      13026         709           24     107.086            23
14303         1      38817         535            3      21.564            17
22301         2      31317         165            1      16.211             5
25731         0      46358         561            1      10.199            18
16820         3       4323         506            1      60.258 

In [135]:
gr1 = train_x.groupby(['client_id'])["amount_rur"].agg({"s1": "sum"})
print(gr1.head())
#gr.set_index('client_id')
joined = train_target.join(gr1)
zero =  joined.groupby('bins').agg({'s1':'mean'})
print(zero.head())
#print(joined[joined["client_id"]==3694].head(20))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


                  s1
client_id           
4          28404.121
6          15720.739
10         34419.365
11         26789.404
13         17337.467
                s1
bins              
0     46951.066337
1     62098.233179
2     28172.588712
3     60398.480530


In [136]:
gr2 = train_x.groupby(['client_id'])["amount_rur"].agg({"s2": "max"})
joined = train_target.join(gr2)
zero =  joined.groupby('bins').agg({'s2':'mean'})
print(zero.head())

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


               s2
bins             
0     2589.771139
1     3578.836228
2     1581.059080
3     3319.801815


In [13]:
gr3 = train_x.groupby(['client_id'])["amount_rur"].agg({"s3": "min"})
joined = train_target.join(gr3)
zero =  joined.groupby('bins').agg({'s3':'mean'})
print(zero.head())

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


            s3
bins          
0     0.294786
1     0.557962
2     0.196993
3     0.396317


In [17]:
grouped_by_group = train_x.groupby(['client_id', 'small_group'], as_index = False)['amount_rur'].agg({'c4':'min'})
gr4  = grouped_by_group.loc[grouped_by_group.groupby('client_id')['c4'].idxmax()][['client_id','small_group']]
gr4.columns = ['client_id', 'c4']
gr4.set_index('client_id', inplace=True)

joined = train_target.join(gr4)
zero =  joined.groupby('bins').agg({'c4':'mean'})
print(zero.head())

             c4
bins           
0     47.302038
1     47.874875
2     44.124609
3     48.889878


In [18]:
grouped_by_group = train_x.groupby(['client_id', 'small_group'], as_index = False)['amount_rur'].agg({'c5':'sum'})
gr5  = grouped_by_group.loc[grouped_by_group.groupby('client_id')['c5'].idxmax()][['client_id','small_group']]
gr5.columns = ['client_id', 'c5']
gr5.set_index('client_id', inplace=True)

joined = train_target.join(gr5)
zero =  joined.groupby('bins').agg({'c5':'mean'})
print(zero.head())

            c5
bins          
0     7.071834
1     5.203390
2     7.260412
3     6.230630


In [137]:
gr6 = train_x.groupby(['client_id']).apply(lambda x: pd.Series({'s6':
                    (x['small_group']==4).sum()}))

#gr.set_index('client_id', inplace=True)
joined = train_target.join(gr6)
zero =  joined.groupby('bins').agg({'s6':'mean'})
print(zero.head())

             s6
bins           
0     36.456231
1     53.253074
2     29.438519
3     41.931091


In [383]:
grr = train_x.groupby(['client_id']).agg(s6=('small_group', lambda x: x[x == 4].sum()))

print(grr.head())
joined = train_target.join(grr)
zero =  joined.groupby('bins').agg({'t':'mean'})
print(zero.head())

[autoreload of pandas failed: Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 378, in superreload
    module = reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\pandas\__init__.py", line 22, in <module>
    from pandas.compat.numpy import (
ImportError: cannot import nam

]
[autoreload of pandas.core.arrays.datetimes failed: Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 378, in superreload
    module = reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\pandas\core\arrays\datetimes.py", line 55, in <module>
    import pandas.core.common a

]
[autoreload of pandas.core.arrays.integer failed: Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 378, in superreload
    module = reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\pandas\core\arrays\integer.py", line 769, in <module>
    "__doc__": _dtype_docstring.fo

]
[autoreload of pandas.core.indexes.datetimes failed: Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 378, in superreload
    module = reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\pandas\core\indexes\datetimes.py", line 21, in <module>
    import pandas.core.dtypes

]
[autoreload of pandas.core.indexes.range failed: Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 378, in superreload
    module = reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\pandas\core\indexes\range.py", line 28, in <module>
    import pandas.core.common as com


]
[autoreload of pandas.plotting._misc failed: Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 378, in superreload
    module = reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\pandas\plotting\_misc.py", line 6, in <module>
    from pandas.plotting._core import _get_plo

]
[autoreload of pandas.core.sparse.series failed: Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 378, in superreload
    module = reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\pandas\core\sparse\series.py", line 25, in <module>
    import pandas.core.ops as ops
Att

]
[autoreload of pandas.core.reshape.pivot failed: Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\IPython\extensions\autoreload.py", line 378, in superreload
    module = reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "C:\Users\user\Anaconda3\envs\kaggle\lib\site-packages\pandas\core\reshape\pivot.py", line 9, in <module>
    import pandas.core.common as com
A

AttributeError: module 'pandas.compat' has no attribute 'string_types'

In [382]:
from importlib.metadata import version
version('pandas')

ModuleNotFoundError: No module named 'importlib.metadata'

In [16]:
grouped = train_x.groupby(['client_id', 'month_number'], as_index=False)

In [100]:

client_per_month_spendings = train_x.groupby(['client_id', 'month_number']).agg({"amount_rur": "sum"}).groupby('client_id').agg({'amount_rur':'mean'})

In [181]:
# Вычислием среднюю максимальную транзакцию по клиентам. Далее найдем среднее отклонение максим транзакций по клиентам. 
# Это позволит найти "большие" транзакции , как mean + std. у нас не гаусово распределение, но все же оперделим значение "большой" транзакции 
# именно так.
grouped_by_max = train_x.groupby(['client_id'], as_index = False).agg({'amount_rur':'max'})
print(grouped_by_max.agg({'amount_rur':'mean'})['amount_rur'] )
large_transactions = grouped_by_max.agg({'amount_rur':'mean'})['amount_rur'] /5
print(large_transactions)

2765.900111078898
553.1800222157797


In [52]:
def prepare_featured_data_v2(df):
    gr_by_client = df.groupby(['client_id'])
    gr1 = gr_by_client["amount_rur"].agg({"s1": "sum"})
    
    gr2 = gr_by_client["amount_rur"].agg({"s2": "max"}) 
    gr3 = gr_by_client["amount_rur"].agg({"s3": "min"})

    gr4_by_group = df.groupby(['client_id', 'small_group'], as_index = False)['amount_rur'].agg({'s4':'min'})
    gr4  = gr4_by_group.loc[gr4_by_group.groupby('client_id')['s4'].idxmax()][['client_id','small_group']]
    gr4.columns = ['client_id', 's4']
    gr4.set_index('client_id', inplace=True)

    
    gr5_by_group = df.groupby(['client_id', 'small_group'], as_index = False)['amount_rur'].agg({'s5':'sum'})
    gr5  = gr5_by_group.loc[gr5_by_group.groupby('client_id')['s5'].idxmax()][['client_id','small_group']]
    gr5.columns = ['client_id', 's5']
    gr5.set_index('client_id', inplace=True)
    
    gr6 = df.groupby(['client_id']).apply(lambda x: pd.Series({'s6':
                    (x['small_group']==4).sum() + 0.000001}))

    gr7 = df.groupby(['client_id']).apply(lambda x: pd.Series({'s7':
                    (x['small_group']==27).sum() + 0.000001}))
    
    gr8 = df.groupby(['client_id']).apply(lambda x: pd.Series({'s8':
                    (x['small_group']==61).sum() + 0.000001}))
    
    gr9 = df.groupby(['client_id']).apply(lambda x: pd.Series({'s9':
                    (x['small_group']==77).sum() + 0.000001}))
    
    gr10 = df.groupby(['client_id']).apply(lambda x: pd.Series({'s10':
                    (x['small_group']==86).sum() + 0.000001}))
    
    gr11 = df.groupby(['client_id'])["trans_date"].agg({"s11": "count"})
    
    res = gr1.join(gr2).join(gr3).join(gr4).join(gr5).join(gr6).join(gr7).join(gr8).join(gr9).join(gr10).join(gr11)
    res.fillna(0, inplace=True)
    return res
    

In [26]:
from sklearn import preprocessing


In [27]:
def normalize(column_names_to_normalize, df):
    x = df[column_names_to_normalize].values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    
    df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = df.index)
    df[column_names_to_normalize] = df_temp
    return df

In [53]:
from torch import nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.hidden0 = nn.Linear(11, 8)
        self.hidden1 = nn.Linear(8, 16)
        self.hidden2 = nn.Linear(16, 32)
        
        self.bn1 = nn.BatchNorm1d(16)
        self.output = nn.Linear(32, 4)

        
    def forward(self, x):
        x = F.relu(self.hidden0(x)) 
        x = F.relu(self.bn1(self.hidden1(x))) 
        x = F.relu(self.hidden2(x))
        return F.log_softmax(self.output(x))
        

In [14]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import torchvision

class Ds(Dataset):
    
    def __init__(self, data, labels):
        self.data = labels.join(data)
        print(self.data.head(30))
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data.iloc[index, 1:]
        y = self.data.iloc[index, 0]

        return torch.tensor(x), torch.tensor(y)

In [54]:
train_y = ds.get_data('train_target')
print(train_y.head())

train_x = ds.get_data('train_x')

#train_y.set_index('client_id', inplace=True)
data = prepare_featured_data_v2(train_x)
print(data.head(10))

processed = normalize(["s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", "s9", "s10", "s11"], data)

train_dataset = Ds(data = processed, labels = train_y)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  

           bins
client_id      
1046          0
34089         2
34848         1
47076         3
10938         2
                  s1        s2     s3  s4  s5         s6        s7         s8  \
client_id                                                                       
4          28404.121  1341.802  0.043   2   1  93.000001  0.000001   0.000001   
6          15720.739   315.781  0.045  80   1  10.000001  0.000001   0.000001   
10         34419.365   654.893  0.045  23   1  65.000001  0.000001   0.000001   
11         26789.404  2105.058  0.388  66  88  23.000001  0.000001   2.000001   
13         17337.467  1190.322  0.043  83  11  45.000001  0.000001   0.000001   
14         26622.629  1167.291  0.043   5   1  39.000001  0.000001   1.000001   
17         58337.998  5692.843  0.028  54   1  34.000001  2.000001   9.000001   
19         46941.495  2049.338  0.259  26   1  98.000001  0.000001  21.000001   
20         79997.006  3213.706  0.043  73  13  10.000001  0.000001   0.000001 

In [29]:
import torch 
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda"),
else:
    device = torch.device("cpu")

In [55]:
import numpy as np
import torch.optim as optim

learning_rate = 0.001


print(device)
network = Net().cuda()
optimizer = optim.Adam(network.parameters(), lr=learning_rate)

for epoch in range(1, 10):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #if batch_idx > 2460:
            #output = network(data.cuda())
            #pred = output.data.max(1, keepdim=True)[1]
            #print('ACURACY: {}'.format((pred.flatten() == target).sum()))
       # else:
            optimizer.zero_grad()
            output = network(data.cuda())
            loss = F.cross_entropy(output, target.cuda()).cuda()
            loss.backward()
            optimizer.step()
    print('Train Epoch: {} \tLoss: {:.6f}'.format(
            epoch, loss.item()))

(device(type='cuda'),)
Train Epoch: 1 	Loss: 1.259120
Train Epoch: 2 	Loss: 1.248440
Train Epoch: 3 	Loss: 1.240419
Train Epoch: 4 	Loss: 1.449114
Train Epoch: 5 	Loss: 1.512479
Train Epoch: 6 	Loss: 0.998040
Train Epoch: 7 	Loss: 1.167227
Train Epoch: 8 	Loss: 1.115762
Train Epoch: 9 	Loss: 1.161456


In [1]:
print(len(res_df[res_df['large_num']>0]))

NameError: name 'res_df' is not defined

In [57]:
valid_y = ds.get_data('valid_target')
#valid_y.set_index('client_id', inplace = True)

valid_x = ds.get_data('valid_x')
valid_x = normalize(["s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", "s9", "s10", "s11"], prepare_featured_data_v2(valid_x))

valid_dataset = Ds(data = valid_x, labels = valid_y)

total = len(valid_dataset)
valid_loader = DataLoader(valid_dataset, batch_size=total)  



           bins        s1        s2        s3        s4        s5        s6  \
client_id                                                                     
24662         2  0.014923  0.004839  0.015675  0.284211  0.066298  0.087302   
47651         3  0.020765  0.003163  0.088254  0.147368  0.005525  0.219577   
36224         2  0.004771  0.002299  0.092549  0.378947  0.005525  0.126984   
21691         1  0.018916  0.017944  0.184883  0.173684  0.071823  0.100529   
29957         3  0.025960  0.015075  0.009019  0.089474  0.011050  0.100529   
30401         2  0.011060  0.002129  0.092549  0.284211  0.005525  0.029101   
15639         0  0.018530  0.002320  0.068499  0.242105  0.005525  0.111111   
28539         1  0.016628  0.002953  0.008804  0.378947  0.005525  0.211640   
39868         3  0.038876  0.010590  0.009019  0.531579  0.132597  0.079365   
48975         0  0.038527  0.008853  0.323599  0.068421  0.005525  0.087302   
26720         0  0.022271  0.006457  0.006657  0.236

In [58]:
def validateNet(net):
    r, idd = iter(valid_loader).next()
    output = net(r.cuda()).cpu()
    
    pred = output.data.max(1, keepdim=True)[1]
   # print(pred)
    print('ACURACY: {}'.format((pred.flatten() == idd).sum()))
    
validateNet(network)

ACURACY: 2776


In [194]:
print(total)

5855
